In [14]:
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
from textblob import TextBlob, Word
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
stemmer = SnowballStemmer('english')

In [5]:
#define functions here

def word_tokenize(text, how = 'lemma'):
    words = TextBlob(text).words
    if how == 'lemma':
        return [word.lemmatize() for word in words]
    elif how == 'stem':
        return [stemmer.stem(word) for word in words]
    
def spelling(text):
    words = TextBlob(text).words
    return [word.correct() for word in words]

In [4]:
#read in all csv files

bio = pd.read_csv('https://raw.githubusercontent.com/sglembocki/Machine-Learning/master/StackExchangeLearning/biology.csv')
cooking = pd.read_csv('https://raw.githubusercontent.com/sglembocki/Machine-Learning/master/StackExchangeLearning/cooking.csv')
crypto = pd.read_csv('https://raw.githubusercontent.com/sglembocki/Machine-Learning/master/StackExchangeLearning/crypto.csv')
diy = pd.read_csv('https://raw.githubusercontent.com/sglembocki/Machine-Learning/master/StackExchangeLearning/diy.csv')
robot = pd.read_csv('https://raw.githubusercontent.com/sglembocki/Machine-Learning/master/StackExchangeLearning/robotics.csv')
travel = pd.read_csv('https://raw.githubusercontent.com/sglembocki/Machine-Learning/master/StackExchangeLearning/travel.csv')

In [16]:
bio['content_decoded'] = bio['content'].str.decode('utf-8', errors = 'ignore').str.lower()
bio.head()

,id,title,content,tags,content_decoded
0,1,What is the criticality of the ribosome bindin...,"<p>In prokaryotic translation, how critical fo...",ribosome binding-sites translation synthetic-b...,"<p>in prokaryotic translation, how critical fo..."
1,2,How is RNAse contamination in RNA based experi...,<p>Does anyone have any suggestions to prevent...,rna biochemistry,<p>does anyone have any suggestions to prevent...
2,3,Are lymphocyte sizes clustered in two groups?,<p>Tortora writes in <em>Principles of Anatomy...,immunology cell-biology hematology,<p>tortora writes in <em>principles of anatomy...
3,4,How long does antibiotic-dosed LB maintain goo...,<p>Various people in our lab will prepare a li...,cell-culture,<p>various people in our lab will prepare a li...
4,5,Is exon order always preserved in splicing?,<p>Are there any cases in which the splicing m...,splicing mrna spliceosome introns exons,<p>are there any cases in which the splicing m...


In [7]:
bio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13196 entries, 0 to 13195
Data columns (total 4 columns):
id         13196 non-null int64
title      13196 non-null object
content    13196 non-null object
tags       13196 non-null object
dtypes: int64(1), object(3)
memory usage: 412.4+ KB


In [8]:
#null accuracy

null = bio.tags.value_counts() / bio.shape[0]
null.head()

evolution        0.012807
human-biology    0.012504
genetics         0.012428
biochemistry     0.008184
cell-biology     0.006896
Name: tags, dtype: float64

In [17]:
x_train, x_test, y_train, y_test = train_test_split(bio.content_decoded, bio.tags, random_state = 1)
x_train.head()

6474     <p>suppose you haven't hit your nails so the w...
6491     <ol>\n<li>a multivalent antibody molecule such...
272      <p>what is known about the targets of golgi st...
11469    <p>antioxidants such as ascorbate and glutathi...
11950    <p>i'm trying to establish if it's required to...
Name: content_decoded, dtype: object

In [18]:
vect = CountVectorizer(stop_words = 'english', analyzer = lambda x: word_tokenize(x, how = 'stem')) #troubleshoot stop_words
x_train_dtm = vect.fit_transform(x_train)

In [19]:
#examine the vocabulary and document-term matrix together

train_arr = x_train_dtm.toarray()
train_features = vect.get_feature_names()
pd.DataFrame(x_train_dtm.toarray(), columns = vect.get_feature_names()).head()

MemoryError: 